In [1]:
import copy
import torch
import torch.nn as nn
import numpy as np
import random
import os
import torch.nn.functional as F
from torch.utils.data import DataLoader
from sklearn.preprocessing import label_binarize
from sklearn import metrics

from sklearn.decomposition import PCA

from flcore.pflniid_utils.data_utils import read_client_data
from utils.custom_loss_class import CPHSLoss
from utils.emg_dataset_class import *

In [2]:
import time
from flcore.pflniid_utils.privacy import *

In [3]:
from flcore.clients.clientbase import Client
from flcore.clients.clientavg import clientAVG
from flcore.servers.serverbase import Server
from flcore.servers.serveravg import FedAvg
from flcore.servers.serverlocal import Local

In [4]:
update_ix = [0,  1200,  2402,  3604,  4806,  6008,  7210,  8412,  9614, 10816, 12018, 13220, 14422, 15624, 16826, 18028, 19230, 20432, 20769]

In [5]:
import argparse
parser = argparse.ArgumentParser()

In [6]:
# general
parser.add_argument('-go', "--goal", type=str, default="test", 
                    help="The goal for this experiment")
parser.add_argument('-dev', "--device", type=str, default="cpu",  # KAI: Changed the default to cpu
                    choices=["cpu", "cuda"])
parser.add_argument('-did', "--device_id", type=str, default="0")
parser.add_argument('-data', "--dataset", type=str, default="cphs")  # KAI: Changed the default to cphs (from mnist)
#parser.add_argument('-nb', "--num_classes", type=int, default=10)  # Not doing classification...
parser.add_argument('-m', "--model", type=str, default="LinearRegression")  # KAI: Changed the default to Linear Regression
parser.add_argument('-lbs', "--batch_size", type=int, default=1200)  # Setting it to a full update would be 1300ish... how many batches does it run? In one epoch? Not even sure where that is set
# The 1300 and the batch size are 2 separate things...
# I want to restrict the given dataset to just the 1300, but then iterate in batches... or do I since we don't have that much data and can probably just use all the data at once? Make batch size match the update size? ...
parser.add_argument('-lr', "--local_learning_rate", type=float, default=0.005,
                    help="Local learning rate")
parser.add_argument('-ld', "--learning_rate_decay", type=bool, default=False)
parser.add_argument('-ldg', "--learning_rate_decay_gamma", type=float, default=0.99)
parser.add_argument('-gr', "--global_rounds", type=int, default=250)  # KAI: Switched to 250 down from 2000
parser.add_argument('-ls', "--local_epochs", type=int, default=1, 
                    help="Multiple update steps in one local epoch.")  # KAI: I think it was 1 originally.  I'm gonna keep it there.  Does this mean I can set batchsize to 1300 and cook?Is my setup capable or running multiple epochs? Implicitly I was doing 1 epoch before, using the full update data I believe...
parser.add_argument('-algo', "--algorithm", type=str, default="FedAvg")
parser.add_argument('-jr', "--join_ratio", type=float, default=0.2,
                    help="Ratio of clients per round")
parser.add_argument('-rjr', "--random_join_ratio", type=bool, default=False,
                    help="Random ratio of clients per round")
parser.add_argument('-nc', "--num_clients", type=int, default=14,
                    help="Total number of clients")
parser.add_argument('-dp', "--privacy", type=bool, default=False,
                    help="differential privacy")
parser.add_argument('-dps', "--dp_sigma", type=float, default=0.0)
parser.add_argument('-sfn', "--save_folder_name", type=str, default='items')

# SECTION: practical
parser.add_argument('-cdr', "--client_drop_rate", type=float, default=0.0,
                    help="Rate for clients that train but drop out")
parser.add_argument('-tsr', "--train_slow_rate", type=float, default=0.0,
                    help="The rate for slow clients when training locally")
parser.add_argument('-ssr', "--send_slow_rate", type=float, default=0.0,
                    help="The rate for slow clients when sending global model")
parser.add_argument('-ts', "--time_select", type=bool, default=False,
                    help="Whether to group and select clients at each round according to time cost")
parser.add_argument('-tth', "--time_threthold", type=float, default=10000,
                    help="The threthold for droping slow clients")

# SECTION: Kai's additional args
parser.add_argument('-pca_channels', "--pca_channels", type=int, default=64,
                    help="Number of principal components. 64 means do not use any PCA")
parser.add_argument('-lambdaF', "--lambdaF", type=float, default=0.0,
                    help="Penalty term for user EMG input (user effort)")
parser.add_argument('-lambdaD', "--lambdaD", type=float, default=1e-3,
                    help="Penalty term for the decoder norm (interface effort)")
parser.add_argument('-lambdaE', "--lambdaE", type=float, default=1e-4,
                    help="Penalty term on performance error norm")
parser.add_argument('-starting_update', "--starting_update", type=int, default=0,
                    help="Which update to start on (for CPHS Simulation). Use 0 or 10.")
parser.add_argument('-test_split_fraction', "--test_split_fraction", type=float, default=0.2,
                    help="Fraction of data to use for testing")
parser.add_argument('-device_channels', "--device_channels", type=int, default=64,
                    help="Number of recording channels with the used EMG device")
parser.add_argument('-dt', "--dt", type=float, default=1/60,
                    help="Delta time, amount of time (sec?) between measurements")
parser.add_argument('-normalize_emg', "--normalize_emg", type=bool, default=False,
                    help="Normalize the input EMG signals")
parser.add_argument('-normalize_V', "--normalize_V", type=bool, default=False,
                    help="Normalize the V term in the cost function")
parser.add_argument('-local_round_threshold', "--local_round_threshold", type=int, default=50,
                    help="Number of communication rounds per client until a client will advance to the next batch of streamed data")
parser.add_argument('-debug_mode', "--debug_mode", type=bool, default=False,
                    help="In debug mode, the code is run to minimize overhead time in order to debug as fast as possible.  Namely, the data is held at the server to decrease init time, and communication delays are ignored.")
parser.add_argument('-condition_number', "--condition_number", type=int, default=1,
                    help="Which condition number (trial) to train on")
parser.add_argument('-test_split_each_update', "--test_split_each_update", type=bool, default=False,
                    help="Implement train/test split within each update or on the entire dataset")
parser.add_argument('-verbose', "--verbose", type=bool, default=False,
                    help="Print out a bunch of extra stuff")
parser.add_argument('-slow_clients_bool', "--slow_clients_bool", type=bool, default=False,
                    help="Control whether or not to have ANY slow clients")
parser.add_argument('-return_cost_func_comps', "--return_cost_func_comps", type=bool, default=False, #True
                    help="Return Loss, Error, DTerm, FTerm from loss class")
parser.add_argument('-test_split_users', "--test_split_users", type=bool, default=False,
                    help="Split testing data by holding out some users (fraction held out determined by test_split_fraction)")
    
parser.add_argument('-t', "--times", type=int, default=1,
                    help="Running times")
parser.add_argument('-ab', "--auto_break", type=bool, default=False)
parser.add_argument('-dlg', "--dlg_eval", type=bool, default=False)
parser.add_argument('-dlgg', "--dlg_gap", type=int, default=100)
parser.add_argument('-bnpc', "--batch_num_per_client", type=int, default=2)  # Only used with DLG
parser.add_argument('-eg', "--eval_gap", type=int, default=1,
                    help="Rounds gap for evaluation")
parser.add_argument('-nnc', "--num_new_clients", type=int, default=0)

# This one for sure breaks it
#parser.add_argument('-fte', "--fine_tuning_epoch", type=int, default=0)

_StoreAction(option_strings=['-nnc', '--num_new_clients'], dest='num_new_clients', nargs=None, const=None, default=0, type=<class 'int'>, choices=None, required=False, help=None, metavar=None)

In [7]:
#args = parser.parse_args()
args = parser.parse_known_args()

args = args[0]
args.fine_tuning_epoch=0
dataset = 'cphs'

In [8]:
time_list = []
#reporter = MemReporter()
model_str = args.model

# Switched args.prev to 0 since it wasn't working
#for i in range(0, args.times):
print(f"\n============= Running time: {0}th =============")
print("Creating server and clients ...")
start = time.time()

# Generate args.model
args.model = torch.nn.Linear(args.pca_channels, 2)  #input_size, output_size

print(args.model)

# select algorithm
if args.algorithm == "FedAvg":
    server = FedAvg(args, 0)
elif args.algorithm == "Local":
    server = Local(args, 0)
else:
    raise NotImplementedError

#server.train()

#time_list.append(time.time()-start)
#print(f"\nAverage time cost: {round(np.average(time_list), 2)}s.")


============= Running time: 0th =============
Creating server and clients ...
Linear(in_features=64, out_features=2, bias=True)
Serveravg init(): set_slow_clients()
Serveravg init(): set_clients()
SBSC: iter 0
SBSC: iter 1
SBSC: iter 2
SBSC: iter 3
SBSC: iter 4
SBSC: iter 5
SBSC: iter 6
SBSC: iter 7
SBSC: iter 8
SBSC: iter 9
SBSC: iter 10
SBSC: iter 11
SBSC: iter 12
SBSC: iter 13

Join ratio / total clients: 0.2 / 14
Finished creating server and clients.


In [9]:
server.selected_clients = server.clients
with torch.no_grad():
    # subscript global_model with [0] if it is sequential instead of linear model --> does that return just the first layer then?
    server.global_model.weight.fill_(0)

#for i in range(self.global_rounds+1):
if 0%server.eval_gap == 0:
    print(f"\n-------------Round number: {0}-------------")
    if 0!=0:
        print("\nEvaluate personalized models")
        server.evaluate()

        #print(f"len: {len(self.rs_train_loss[-1])}")
        if type(server.rs_train_loss[-1]) in [int, float]:
            print(f"rs_train_loss: {server.rs_train_loss[-1]}")
        else:
            print(f"len: {len(server.rs_train_loss[-1])}")
        print()

server.selected_clients = server.select_clients()
print(f"Selected client IDs: {[client.ID for client in server.selected_clients]}")


-------------Round number: 0-------------
Selected client IDs: [6, 11]


In [10]:
#print("CLIENT TRAINING")
#for client in server.selected_clients:
#    client.train()
#    print(f"Client{client.ID} loss: {client.loss_log[-1]:0,.3f}")

my_client = server.selected_clients[0]

In [11]:
my_client.local_epochs

1

In [12]:
#def train(self):
trainloader = my_client.load_train_data()
# self.model.to(self.device)
my_client.model.train()

# differential privacy
#if self.privacy:
#    self.model, self.optimizer, trainloader, privacy_engine = \
#        initialize_dp(self.model, self.optimizer, trainloader, self.dp_sigma)

start_time = time.time()

max_local_steps = my_client.local_epochs
#if self.train_slow:
#    max_local_steps = np.random.randint(1, max_local_steps // 2)

## Default Trainloader

In [13]:
for i, (x, y) in enumerate(trainloader):
    print(f"Batch {i}: x has size {x.size()}; y has size {y.size()}")

Batch 0: x has size torch.Size([1200, 64]); y has size torch.Size([1200, 2])


In [14]:
i=0
base_data_path = 'C:\\Users\\kdmen\\Desktop\\Research\\personalization-privacy-risk\\Data\\Client_Specific_Files\\'
client = clientAVG(server.args, 
                    ID=i, 
                    train_samples = base_data_path + "UserID" + str(i) + "_TrainData_8by20770by64.npy", 
                    test_samples = base_data_path + "UserID" + str(i) + "_Labels_8by20770by2.npy", 
                    train_slow=False, 
                    send_slow=False)

In [15]:
my_client.test_split_users

False

In [16]:
my_client.test_split_each_update

False

In [118]:
print(my_client.ID)
my_client.condition_number = 0

6


In [121]:
# FROM CLIENTBASE.PY

#def load_train_data(self, batch_size=None):
batch_size=None
# Load full client dataasets
#if my_client.local_round == 0:

###########################################################################################
#self._load_train_data()   # Returns nothing, sets self variables
# Load in client's data
with open(my_client.samples_path, 'rb') as handle:
    samples_npy = np.load(handle)
    print(samples_npy[0,:10,0])
with open(my_client.labels_path, 'rb') as handle:
    labels_npy = np.load(handle)
# Select for given condition #THIS IS THE ACTUAL TRAINING DATA AND LABELS FOR THE GIVEN TRIAL
my_client.cond_samples_npy = samples_npy[my_client.condition_number,:,:]
print(my_client.cond_samples_npy[:10,0])
my_client.cond_labels_npy = labels_npy[my_client.condition_number,:,:]
# Split data into train and test sets
testsplit_upper_bound = round((1-my_client.test_split_fraction)*(my_client.cond_samples_npy.shape[0]))
# Set the number of examples (used to be done on init) --> ... THIS IS ABOUT TRAIN/TEST SPLIT
my_client.train_samples = testsplit_upper_bound
my_client.test_samples = my_client.cond_samples_npy.shape[0] - testsplit_upper_bound
train_test_update_number_split = min(my_client.update_ix, key=lambda x:abs(x-testsplit_upper_bound))
my_client.max_training_update_upbound = my_client.update_ix.index(train_test_update_number_split)
###########################################################################################

# Why is this in local_round=0...
#if my_client.current_update < my_client.max_training_update_upbound:
#    my_client.update_lower_bound = my_client.update_ix[my_client.current_update]
#    my_client.update_upper_bound = my_client.update_ix[my_client.current_update+1]
#else:
#    ...
# I just added this, should really be idx bound not update bound...
my_client.update_lower_bound = my_client.update_ix[my_client.current_update]
my_client.update_upper_bound = my_client.update_ix[my_client.current_update+1]

[10.42677364 14.68297396 14.68297396 16.38068939 16.38068939 16.38068939
 16.59242395  9.08707751  9.08707751  9.08707751]
[10.42677364 14.68297396 14.68297396 16.38068939 16.38068939 16.38068939
 16.59242395  9.08707751  9.08707751  9.08707751]


In [18]:
print(f"update lower bound: {my_client.update_lower_bound}")
print(f"update upper bound: {my_client.update_upper_bound}")

update lower bound: 0
update upper bound: 1200


In [19]:
my_client.local_round += 1
# Check if you need to advance the update
# ---> THIS IMPLIES THAT I AM CREATING A NEW TRAINING LOADER FOR EACH UPDATE...
# Uh why is 16 hardcoded...
# This is the update logic
if (my_client.local_round>1) and (my_client.current_update < 16) and (my_client.local_round%my_client.local_round_threshold==0):
    my_client.current_update += 1
    print(f"Client{my_client.ID} advances to update {my_client.current_update}")
    # Slice the full client dataset based on the current update number
    if my_client.current_update < my_client.max_training_update_upbound:
        my_client.update_lower_bound = my_client.update_ix[my_client.current_update]
        my_client.update_upper_bound = my_client.update_ix[my_client.current_update+1]
    else:
        my_client.update_lower_bound = my_client.max_training_update_upbound - 1
        my_client.update_upper_bound = my_client.max_training_update_upbound

In [20]:
# Set the Dataset Obj
# Uhhhh is this creating a new one each time? As long as its not re-reading in the data it probably doesn't matter...
#train_data = read_client_data(self.dataset, self.ID, self.current_update, is_train=True)  # Original code
#CustomEMGDataset(emgs_block1[my_user][condition_idx,update_lower_bound:update_upper_bound,:], refs_block1[my_user][condition_idx,update_lower_bound:update_upper_bound,:])
training_dataset_obj = CustomEMGDataset(my_client.cond_samples_npy[my_client.update_lower_bound:my_client.update_upper_bound,:], my_client.cond_labels_npy[my_client.update_lower_bound:my_client.update_upper_bound,:])
X_data = torch.Tensor(training_dataset_obj['x']).type(torch.float32)
y_data = torch.Tensor(training_dataset_obj['y']).type(torch.float32)
training_data_for_dataloader = [(x, y) for x, y in zip(X_data, y_data)]

In [21]:
training_dataset_obj = CustomEMGDataset(my_client.cond_samples_npy[my_client.update_lower_bound:my_client.update_upper_bound,:], my_client.cond_labels_npy[my_client.update_lower_bound:my_client.update_upper_bound,:])

In [22]:
my_client.cond_samples_npy[my_client.update_lower_bound:my_client.update_upper_bound,:].shape

(1200, 64)

In [23]:
my_client.cond_labels_npy[my_client.update_lower_bound:my_client.update_upper_bound,:].shape

(1200, 2)

In [24]:
torch.Tensor(training_dataset_obj['x']).type(torch.float32).shape

torch.Size([1200, 64])

In [25]:
len(training_data_for_dataloader)

1200

In [26]:
len(training_data_for_dataloader[0])

2

That all looks fine...

In [27]:
# Set dataloader
if batch_size == None:
    batch_size = my_client.batch_size
trainloader = DataLoader(
    dataset=training_data_for_dataloader,
    batch_size=batch_size, 
    drop_last=False,  # Yah idk if this should be true or false or if it matters...
    shuffle=False) 
#return dl

In [28]:
my_client.cond_samples_npy.shape

(20770, 64)

In [29]:
my_client.cond_labels_npy.shape

(20770, 2)

In [30]:
for i, (x, y) in enumerate(trainloader):
    print(f"Batch {i}: x has size {x.size()}; y has size {y.size()}")

Batch 0: x has size torch.Size([1200, 64]); y has size torch.Size([1200, 2])


In [31]:
1200*15+370

18370

It shouldn't be going through all the batches at once right... it should only be going through the first updates worth...
- There's 16 batches, but 18 updates... looks like it held the last few batches out for testing? 
- Don't really wanna test on the actual last batch
- Also shouldn't the training updates by broken up perfectly and not with some leftover? Code isn't working as expected...
- Does simulate data streaming have any affect? NO
- Where was it in the code that I kept auto-remaking trainloaders?

In [32]:
len(update_ix)-1

18

Uhh is this code only used once lol
- It's literaly only used in the client init...

In [33]:
# Before this I need to run the INIT update segmentation code...
#init_dl = self.load_train_data()
#self.simulate_data_streaming(init_dl)
# ^ This func sets F, V, etc

## Load Testing Data
update setting code SHOULD NOT be in test data (unless each update has its own separate test data)

CORRECTION TO LOAD_TEST_DATA()

In [38]:
# CORRECTED
#def load_test_data(self, batch_size=None):
batch_size=None
# Make sure this runs AFTER load_train_data so the data is already loaded in
if batch_size == None:
    batch_size = my_client.batch_size

##########################################################################
# Idk if update_ix would need self or if it even exists within the object yet lmao
my_client.test_split_idx = update_ix[my_client.max_training_update_upbound]
##########################################################################
    
#test_data = read_client_data(self.dataset, self.ID, self.current_update, is_train=False)
testing_dataset_obj = CustomEMGDataset(my_client.cond_samples_npy[my_client.test_split_idx:,:], my_client.cond_labels_npy[my_client.test_split_idx:,:])
X_data = torch.Tensor(testing_dataset_obj['x']).type(torch.float32)
y_data = torch.Tensor(testing_dataset_obj['y']).type(torch.float32)
testing_data_for_dataloader = [(x, y) for x, y in zip(X_data, y_data)]

correctedtestloader = DataLoader(
    dataset=testing_data_for_dataloader,
    batch_size=batch_size, 
    drop_last=False,  # Yah idk if this should be true or false or if it matters...
    shuffle=False) 
#return dl

In [39]:
for i, (x, y) in enumerate(correctedtestloader):
    print(f"Batch {i}: x has size {x.size()}; y has size {y.size()}")

Batch 0: x has size torch.Size([1200, 64]); y has size torch.Size([1200, 2])
Batch 1: x has size torch.Size([1200, 64]); y has size torch.Size([1200, 2])
Batch 2: x has size torch.Size([1200, 64]); y has size torch.Size([1200, 2])
Batch 3: x has size torch.Size([344, 64]); y has size torch.Size([344, 2])


Seems like it fixed it... 
- Do I wanna just drop the last batch? It might mess things up since it's not the same size
- Idk what would happen in real-time trials though if everything has to be set up into uniform sized buckets...

In [40]:
assert(False)

AssertionError: 

In [44]:
for i, (x, y) in enumerate(trainloader):
    print("Resetting x, y back to training data!")

Resetting x, y back to training data!


In [45]:
x.shape

torch.Size([1200, 64])

In [46]:
y.shape

torch.Size([1200, 2])

In [48]:
# WHICH OF THESE LOOPS IS EQUIVALENT TO MY EPOCHS...
running_num_samples = 0
step = 0
#for step in range(max_local_steps):  # I'm assuming this is gradient steps?... are local epochs the same as gd steps?
#for i, (x, y) in enumerate(trainloader):  # This is all the data in a given batch, I think? Can I just kill this... PITA
print(f"Step {step}, pair {i} in traindl: x.size(): {x.size()}")
if type(x) == type([]):
    x[0] = x[0].to(my_client.device)
else:
    x = x.to(my_client.device)
y = y.to(my_client.device)
#if self.train_slow:
#    time.sleep(0.1 * np.abs(np.random.rand()))
output = my_client.model(x)
#print(f"clientAVG ----> Training LOSS {i}")  # What is this even tellimg me lol
loss = my_client.loss(output, y, my_client.model)
t1 = my_client.loss.term1_error.item()
t2 = my_client.loss.term2_ld_decnorm.item()
t3 = my_client.loss.term3_lf_emgnorm.item()
if np.isnan(t1):
    print("CLIENTAVG: Error term is None...")
    t1 = -1
if np.isnan(t2):
    print("CLIENTAVG: Decoder Effort term is None...")
    t2 = -1
if np.isnan(t3):
    print("CLIENTAVG: User Effort term is None...")
    t3 = -1
#my_client.cost_func_comps_log.append((t1, t2, t3))

#my_client.train_time_cost['num_rounds'] += 1
#my_client.train_time_cost['total_cost'] += time.time() - start_time

Step 0, pair 0 in traindl: x.size(): torch.Size([1200, 64])


In [54]:
print(f"output: {output}")
print()
print(f"output - y: {output - y}")
print()
print(f"sum of differences between output and y [sum over 1200 samples, 2 cols]: {sum(output - y)}")
print(f"t1: {t1}")
print(f"t2: {t2}")

output: tensor([[  0.0307,   0.0888],
        [  0.0307,   0.0888],
        [ -0.3059,   0.6793],
        ...,
        [-16.1558,   0.5281],
        [-20.8682,  -0.0383],
        [-20.8682,  -0.0383]], grad_fn=<AddmmBackward0>)

output - y: tensor([[ 3.0684e-02,  8.8819e-02],
        [ 3.0684e-02,  8.8819e-02],
        [-3.0591e-01,  6.7933e-01],
        ...,
        [-3.5156e+01, -9.4719e+00],
        [-3.9868e+01, -1.0038e+01],
        [-3.9868e+01, -1.0038e+01]], grad_fn=<SubBackward0>)

sum of differences between output and y [sum over 1200 samples, 2 cols]: tensor([-21983.8281,  11366.1172], grad_fn=<AddBackward0>)
t1: 65.34932708740234
t2: 0.0006846926989965141


In [55]:
weight_grad = my_client.model.weight.grad
if weight_grad == None:
    print("Weight gradient is None...")
    my_client.gradient_norm_log.append(-1)
else:
    #grad_norm = torch.linalg.norm(self.model.weight.grad, ord='fro')
    grad_norm = np.linalg.norm(my_client.model.weight.grad.detach().numpy())
    my_client.gradient_norm_log.append(grad_norm)
#my_client.loss_log.append(loss.item())
#running_num_samples += x.size(0)
my_client.optimizer.zero_grad()
loss.backward()
my_client.optimizer.step()

Weight gradient is None...


In [56]:
print(f"output: {output}")
print()
print(f"output - y: {output - y}")
print()
print(f"sum of differences between output and y [sum over 1200 samples, 2 cols]: {sum(output - y)}")
print(f"t1: {t1}")
print(f"t2: {t2}")

output: tensor([[  0.0307,   0.0888],
        [  0.0307,   0.0888],
        [ -0.3059,   0.6793],
        ...,
        [-16.1558,   0.5281],
        [-20.8682,  -0.0383],
        [-20.8682,  -0.0383]], grad_fn=<AddmmBackward0>)

output - y: tensor([[ 3.0684e-02,  8.8819e-02],
        [ 3.0684e-02,  8.8819e-02],
        [-3.0591e-01,  6.7933e-01],
        ...,
        [-3.5156e+01, -9.4719e+00],
        [-3.9868e+01, -1.0038e+01],
        [-3.9868e+01, -1.0038e+01]], grad_fn=<SubBackward0>)

sum of differences between output and y [sum over 1200 samples, 2 cols]: tensor([-21983.8281,  11366.1172], grad_fn=<AddBackward0>)
t1: 65.34932708740234
t2: 0.0006846926989965141


In [57]:
print(f"Step {step}, pair {i} in traindl: x.size(): {x.size()}")
if type(x) == type([]):
    x[0] = x[0].to(my_client.device)
else:
    x = x.to(my_client.device)
y = y.to(my_client.device)
#if self.train_slow:
#    time.sleep(0.1 * np.abs(np.random.rand()))
output = my_client.model(x)
#print(f"clientAVG ----> Training LOSS {i}")  # What is this even tellimg me lol
loss = my_client.loss(output, y, my_client.model)
t1 = my_client.loss.term1_error.item()
t2 = my_client.loss.term2_ld_decnorm.item()
t3 = my_client.loss.term3_lf_emgnorm.item()
if np.isnan(t1):
    print("CLIENTAVG: Error term is None...")
    t1 = -1
if np.isnan(t2):
    print("CLIENTAVG: Decoder Effort term is None...")
    t2 = -1
if np.isnan(t3):
    print("CLIENTAVG: User Effort term is None...")
    t3 = -1
    
print(f"output: {output}")
print()
print(f"output - y: {output - y}")
print()
print(f"sum of differences between output and y [sum over 1200 samples, 2 cols]: {sum(output - y)}")
print(f"t1: {t1}")
print(f"t2: {t2}")

Step 0, pair 0 in traindl: x.size(): torch.Size([1200, 64])
output: tensor([[ 5.5665e-02,  7.3136e-02],
        [ 5.5665e-02,  7.3136e-02],
        [ 7.6774e+01, -4.8346e+01],
        ...,
        [ 7.5202e+02, -4.6749e+02],
        [ 9.3611e+02, -5.8086e+02],
        [ 9.3611e+02, -5.8086e+02]], grad_fn=<AddmmBackward0>)

output - y: tensor([[ 5.5665e-02,  7.3136e-02],
        [ 5.5665e-02,  7.3136e-02],
        [ 7.6774e+01, -4.8346e+01],
        ...,
        [ 7.3302e+02, -4.7749e+02],
        [ 9.1711e+02, -5.9086e+02],
        [ 9.1711e+02, -5.9086e+02]], grad_fn=<SubBackward0>)

sum of differences between output and y [sum over 1200 samples, 2 cols]: tensor([1099143.0000, -702366.3750], grad_fn=<AddBackward0>)
t1: 231167.40625
t2: 0.036918941885232925


In [58]:
weight_grad = my_client.model.weight.grad
if weight_grad == None:
    print("Weight gradient is None...")
    my_client.gradient_norm_log.append(-1)
else:
    #grad_norm = torch.linalg.norm(self.model.weight.grad, ord='fro')
    grad_norm = np.linalg.norm(my_client.model.weight.grad.detach().numpy())
    my_client.gradient_norm_log.append(grad_norm)
#my_client.loss_log.append(loss.item())
#running_num_samples += x.size(0)
my_client.optimizer.zero_grad()
loss.backward()
my_client.optimizer.step()

In [59]:
print(f"output: {output}")
print()
print(f"output - y: {output - y}")
print()
print(f"sum of differences between output and y [sum over 1200 samples, 2 cols]: {sum(output - y)}")
print(f"t1: {t1}")
print(f"t2: {t2}")

output: tensor([[ 5.5665e-02,  7.3136e-02],
        [ 5.5665e-02,  7.3136e-02],
        [ 7.6774e+01, -4.8346e+01],
        ...,
        [ 7.5202e+02, -4.6749e+02],
        [ 9.3611e+02, -5.8086e+02],
        [ 9.3611e+02, -5.8086e+02]], grad_fn=<AddmmBackward0>)

output - y: tensor([[ 5.5665e-02,  7.3136e-02],
        [ 5.5665e-02,  7.3136e-02],
        [ 7.6774e+01, -4.8346e+01],
        ...,
        [ 7.3302e+02, -4.7749e+02],
        [ 9.1711e+02, -5.9086e+02],
        [ 9.1711e+02, -5.9086e+02]], grad_fn=<SubBackward0>)

sum of differences between output and y [sum over 1200 samples, 2 cols]: tensor([1099143.0000, -702366.3750], grad_fn=<AddBackward0>)
t1: 231167.40625
t2: 0.036918941885232925


In [60]:
print(f"Step {step}, pair {i} in traindl: x.size(): {x.size()}")
if type(x) == type([]):
    x[0] = x[0].to(my_client.device)
else:
    x = x.to(my_client.device)
y = y.to(my_client.device)
#if self.train_slow:
#    time.sleep(0.1 * np.abs(np.random.rand()))
output = my_client.model(x)
#print(f"clientAVG ----> Training LOSS {i}")  # What is this even tellimg me lol
loss = my_client.loss(output, y, my_client.model)
t1 = my_client.loss.term1_error.item()
t2 = my_client.loss.term2_ld_decnorm.item()
t3 = my_client.loss.term3_lf_emgnorm.item()
if np.isnan(t1):
    print("CLIENTAVG: Error term is None...")
    t1 = -1
if np.isnan(t2):
    print("CLIENTAVG: Decoder Effort term is None...")
    t2 = -1
if np.isnan(t3):
    print("CLIENTAVG: User Effort term is None...")
    t3 = -1
    
weight_grad = my_client.model.weight.grad
if weight_grad == None:
    print("Weight gradient is None...")
    my_client.gradient_norm_log.append(-1)
else:
    grad_norm = np.linalg.norm(my_client.model.weight.grad.detach().numpy())
    my_client.gradient_norm_log.append(grad_norm)
my_client.optimizer.zero_grad()
loss.backward()
my_client.optimizer.step()
    
print(f"output: {output}")
print()
print(f"output - y: {output - y}")
print()
print(f"sum of differences between output and y [sum over 1200 samples, 2 cols]: {sum(output - y)}")
print(f"t1: {t1}")
print(f"t2: {t2}")

Step 0, pair 0 in traindl: x.size(): torch.Size([1200, 64])
output: tensor([[-1.4554e+00,  1.0354e+00],
        [-1.4554e+00,  1.0354e+00],
        [-4.7664e+03,  3.0408e+03],
        ...,
        [-4.7783e+04,  3.0324e+04],
        [-5.9583e+04,  3.7796e+04],
        [-5.9583e+04,  3.7796e+04]], grad_fn=<AddmmBackward0>)

output - y: tensor([[-1.4554e+00,  1.0354e+00],
        [-1.4554e+00,  1.0354e+00],
        [-4.7664e+03,  3.0408e+03],
        ...,
        [-4.7802e+04,  3.0314e+04],
        [-5.9602e+04,  3.7786e+04],
        [-5.9602e+04,  3.7786e+04]], grad_fn=<SubBackward0>)

sum of differences between output and y [sum over 1200 samples, 2 cols]: tensor([-69182904.,  44109432.], grad_fn=<AddBackward0>)
t1: 908618048.0
t2: 142.64292907714844


# ALTERNATIVE VERSION

In [64]:
# our model
model = torch.nn.Linear(64, 2)

# Construct our loss function and an Optimizer. The call to model.parameters()
# in the SGD constructor will contain the learnable parameters of the two
# nn.Linear modules which are members of the model.

criterion = torch.nn.MSELoss(reduction='sum')

#optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
optimizer = torch.optim.SGD(model.parameters(), lr=0.005)
# Making LR tiny slows it down but it still blows up...

# Training loop
for epoch in range(10):
    for i, (x_data, y_data) in enumerate(trainloader):
        # 1) Forward pass: Compute predicted y by passing x to the model
        y_pred = model(x_data)

        # 2) Compute and print loss
        loss = criterion(y_pred, y_data)
        print(f'Epoch: {epoch} | Loss: {loss.item()} ')

        # Zero gradients, perform a backward pass, and update the weights.
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

Epoch: 0 | Loss: 677238.75 
Epoch: 1 | Loss: 103747800.0 
Epoch: 2 | Loss: 829558685696.0 
Epoch: 3 | Loss: 7786028383338496.0 
Epoch: 4 | Loss: 7.361807371662577e+19 
Epoch: 5 | Loss: 6.962956296650413e+23 
Epoch: 6 | Loss: 6.585809935826766e+27 
Epoch: 7 | Loss: 6.229092318903552e+31 
Epoch: 8 | Loss: 5.891698516480374e+35 
Epoch: 9 | Loss: inf 


In [128]:
#refs_block1, _, _, _, emgs_block1, _, _, _, _, _, _ = pickle.load(handle)
print(emgs_block1[keys[0]].shape)
print(refs_block1[keys[0]].shape)

(8, 20770, 64)
(8, 20770, 2)


In [130]:
cli0_cond1_samples = emgs_block1[keys[0]][0, :, :]
cli0_cond1_labels = refs_block1[keys[0]][0, :, :]

In [139]:
# our model
model = torch.nn.Linear(64, 2)

# Construct our loss function and an Optimizer. The call to model.parameters()
# in the SGD constructor will contain the learnable parameters of the two
# nn.Linear modules which are members of the model.

criterion = torch.nn.MSELoss(reduction='sum')

#optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
optimizer = torch.optim.SGD(model.parameters(), lr=0.00001)
# Making LR tiny slows it down but it still blows up...

x_data = torch.tensor(cli0_cond1_samples, dtype=torch.float32)
y_data = torch.tensor(cli0_cond1_labels, dtype=torch.float32)

# Training loop
for epoch in range(10):
    #for i, (x_data, y_data) in enumerate(trainloader):
    
    # 1) Forward pass: Compute predicted y by passing x to the model
    y_pred = model(x_data)

    # 2) Compute and print loss
    loss = criterion(y_pred, y_data)
    print(f'Epoch: {epoch} | Loss: {loss.item()} ')

    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

Epoch: 0 | Loss: 26502288.0 
Epoch: 1 | Loss: 3441264699637760.0 
Epoch: 2 | Loss: 1.2555818147174948e+24 
Epoch: 3 | Loss: 4.582583999763409e+32 
Epoch: 4 | Loss: inf 
Epoch: 5 | Loss: inf 
Epoch: 6 | Loss: inf 
Epoch: 7 | Loss: inf 
Epoch: 8 | Loss: inf 
Epoch: 9 | Loss: nan 


In [140]:
for i in range(len(keys)):
    print(f"CLIENT{i}")

    # our model
    model = torch.nn.Linear(64, 2)

    # Construct our loss function and an Optimizer. The call to model.parameters()
    # in the SGD constructor will contain the learnable parameters of the two
    # nn.Linear modules which are members of the model.

    criterion = torch.nn.MSELoss(reduction='sum')

    #optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
    optimizer = torch.optim.SGD(model.parameters(), lr=0.00001)
    # Making LR tiny slows it down but it still blows up...

    x_data = torch.tensor(emgs_block1[keys[i]][0, :, :], dtype=torch.float32)
    y_data = torch.tensor(refs_block1[keys[i]][0, :, :], dtype=torch.float32)

    # Training loop
    for epoch in range(10):
        #for i, (x_data, y_data) in enumerate(trainloader):

        # 1) Forward pass: Compute predicted y by passing x to the model
        y_pred = model(x_data)

        # 2) Compute and print loss
        loss = criterion(y_pred, y_data)
        print(f'Epoch: {epoch} | Loss: {loss.item()} ')

        # Zero gradients, perform a backward pass, and update the weights.
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print()

CLIENT0
Epoch: 0 | Loss: 24290276.0 
Epoch: 1 | Loss: 2936180303724544.0 
Epoch: 2 | Loss: 1.0715106186900139e+24 
Epoch: 3 | Loss: 3.910772509543822e+32 
Epoch: 4 | Loss: inf 
Epoch: 5 | Loss: inf 
Epoch: 6 | Loss: inf 
Epoch: 7 | Loss: inf 
Epoch: 8 | Loss: inf 
Epoch: 9 | Loss: nan 

CLIENT1
Epoch: 0 | Loss: 31997452.0 
Epoch: 1 | Loss: 7.60232833712128e+16 
Epoch: 2 | Loss: 4.857631661008698e+26 
Epoch: 3 | Loss: 3.1054302270356055e+36 
Epoch: 4 | Loss: inf 
Epoch: 5 | Loss: inf 
Epoch: 6 | Loss: inf 
Epoch: 7 | Loss: inf 
Epoch: 8 | Loss: nan 
Epoch: 9 | Loss: nan 

CLIENT2
Epoch: 0 | Loss: 16203730.0 
Epoch: 1 | Loss: 3074357198848.0 
Epoch: 2 | Loss: 5.72363532173679e+19 
Epoch: 3 | Loss: 1.360310131660171e+27 
Epoch: 4 | Loss: 3.2437940919801105e+34 
Epoch: 5 | Loss: inf 
Epoch: 6 | Loss: inf 
Epoch: 7 | Loss: inf 
Epoch: 8 | Loss: inf 
Epoch: 9 | Loss: inf 

CLIENT3
Epoch: 0 | Loss: 328153216.0 
Epoch: 1 | Loss: 1.1020855405161585e+20 
Epoch: 2 | Loss: 4.0373252821927207e+31 


I guess try and train an sklearn model on the data? No idea what's going on...

In [142]:
from sklearn.linear_model import LinearRegression

In [146]:
x_data = torch.tensor(emgs_block1[keys[0]][0, :testsplit_upper_bound, :], dtype=torch.float32)
y_data = torch.tensor(refs_block1[keys[0]][0, :testsplit_upper_bound, :], dtype=torch.float32)

reg = LinearRegression().fit(x_data, y_data)
reg.score(x_data, y_data)

0.06147221139504461

In [150]:
print(reg.predict(x_data[:3,:]))
print()
print(y_data[:3,:])

[[-2.800249    0.23163907]
 [-2.800249    0.23163907]
 [-2.800249    0.23163907]]

tensor([[-8.5753e-05, -5.6882e-04],
        [-8.5753e-05, -5.6882e-04],
        [-2.0100e-04, -1.3241e-03]])


In [145]:
x_data.shape

torch.Size([20770, 64])

Even linear regression using sklearn doesn't work?????

In [ ]:
CPHSLoss

In [123]:
import torch
from math import isnan

class TroubleShooting_Loss(torch.nn.modules.loss._Loss):
    def __init__(self, lambdaF=0, lambdaD=1e-3, lambdaE=1e-6, Nd=2, Ne=64, return_cost_func_comps=False, verbose=False, dt=1/60, normalize_V=False) -> None:
        super().__init__()
        self.lambdaF = lambdaF
        self.lambdaD = lambdaD
        self.lambdaE = lambdaE
        self.Nd = Nd
        self.Ne = Ne
        # Don't use return_cost_func_comps since I don't think loss.item() will return a tuple, it only returns scalars AFAIK
        self.dt = dt
        self.normalize_V = normalize_V
        self.verbose = verbose
        self.term1_error = 0
        self.term2_ld_decnorm = 0
        self.term3_lf_emgnorm = 0


    def forward(self, outputs, targets):
        outputs = torch.transpose(outputs, 0, 1)
        p_reference = torch.transpose(targets, 0, 1)
        p_actual = torch.cumsum(outputs, dim=1)*self.dt  # Numerical integration of v_actual to get p_actual
        self.V = (p_reference - p_actual)*self.dt
        if self.normalize_V:
            self.V = self.V/torch.linalg.norm(self.V, ord='fro')
            assert (torch.linalg.norm(self.V, ord='fro')<1.2) and (torch.linalg.norm(self.V, ord='fro')>0.8)
        Vplus = self.V[:,1:]
        # Performance
        return self.lambdaE*(torch.linalg.matrix_norm(outputs[:,:-1] - Vplus)**2)

        
    

In [125]:
my_loss_func = TroubleShooting_Loss()
my_loss_func[0]

TypeError: 'TroubleShooting_Loss' object is not subscriptable

# ITS GOTTA BE THE TRAININGLOADER...

In [66]:
import pickle

In [67]:
with open(r"C:\Users\kdmen\Desktop\Research\personalization-privacy-risk\Data\continuous_full_data_block1.pickle", 'rb') as handle:
    #refs_block1, poss_block1, dec_vels_block1, int_vel_block1, emgs_block1, Ws_block1, Hs_block1, alphas_block1, pDs_block1, times_block1, conditions_block1 = pickle.load(handle)
    refs_block1, _, _, _, emgs_block1, _, _, _, _, _, _ = pickle.load(handle)
    
keys = ['METACPHS_S106', 'METACPHS_S107', 'METACPHS_S108', 'METACPHS_S109', 'METACPHS_S110', 'METACPHS_S111', 'METACPHS_S112', 'METACPHS_S113', 'METACPHS_S114', 'METACPHS_S115', 'METACPHS_S116', 'METACPHS_S117', 'METACPHS_S118', 'METACPHS_S119']
key_to_num = dict()
num_to_key = dict()
for idx, key in enumerate(keys):
    key_to_num[key] = idx
    num_to_key[idx] = key

num_conds = 8
num_channels = 64
num_updates = 19
cphs_starting_update = 10
update_ix = [0,  1200,  2402,  3604,  4806,  6008,  7210,  8412,  9614, 10816, 12018, 13220, 14422, 15624, 16826, 18028, 19230, 20432, 20769]

In [72]:
my_client_num = 6

In [73]:
print(f"EMG Input shape: {emgs_block1[keys[my_client_num]].shape}")
print(f"2D Velocity Label shape: {refs_block1[keys[my_client_num]].shape}")

EMG Input shape: (8, 20770, 64)
2D Velocity Label shape: (8, 20770, 2)


In [74]:
print("Condition 1")
F1tens_full = torch.from_numpy(emgs_block1[keys[my_client_num]][0, :, :])
PREF1tens_full = torch.from_numpy(refs_block1[keys[my_client_num]][0, :, :])

Condition 1


VS

In [70]:
# FROM CLIENTBASE.PY

#def load_train_data(self, batch_size=None):
batch_size=None
# Load full client dataasets
###########################################################################################
#self._load_train_data()   # Returns nothing, sets self variables
# Load in client's data
with open(my_client.samples_path, 'rb') as handle:
    samples_npy = np.load(handle)
with open(my_client.labels_path, 'rb') as handle:
    labels_npy = np.load(handle)


In [71]:
print(my_client.samples_path)

C:\Users\kdmen\Desktop\Research\personalization-privacy-risk\Data\Client_Specific_Files\UserID6_TrainData_8by20770by64.npy


In [75]:
samples_npy.shape

(8, 20770, 64)

In [80]:
emgs_block1[keys[my_client_num]].shape

(8, 20770, 64)

Check differences between samples

In [84]:
cli6_cond0_npy = samples_npy[0]
cli6_cond0_emg = emgs_block1[keys[my_client_num]][0]
sum(sum(cli6_cond0_npy - cli6_cond0_emg))

0.0

Check differences between labels

In [85]:
cli6_cond0_labels_npy = labels_npy[0]
cli6_cond0_ref = refs_block1[keys[my_client_num]][0]
sum(sum(cli6_cond0_labels_npy - cli6_cond0_ref))

0.0

Do train-test split

In [88]:
print(cli6_cond0_npy.shape[0])
testsplit_upper_bound = round((1-my_client.test_split_fraction)*(cli6_cond0_npy.shape[0]))
# Set the number of examples (used to be done on init) --> ... THIS IS ABOUT TRAIN/TEST SPLIT
my_client.train_samples = testsplit_upper_bound
print(my_client.train_samples)
my_client.test_samples = cli6_cond0_npy.shape[0] - testsplit_upper_bound
print(my_client.test_samples)
train_test_update_number_split = min(my_client.update_ix, key=lambda x:abs(x-testsplit_upper_bound))
print(train_test_update_number_split)
my_client.max_training_update_upbound = my_client.update_ix.index(train_test_update_number_split)
print(my_client.max_training_update_upbound)


20770
16616
4154
16826
14


Check out train loader

In [92]:
#def load_train_data(self, batch_size=None):
    # Load full client dataasets
    
#_load_train_data()
#############################################################################
#print(f"Client{self.ID} loading data file in [SHOULD ONLY RUN ONCE PER CLIENT]")
## Load in client's data
#with open(self.samples_path, 'rb') as handle:
#    samples_npy = np.load(handle)
#with open(self.labels_path, 'rb') as handle:
#    labels_npy = np.load(handle)
## Select for given condition #THIS IS THE ACTUAL TRAINING DATA AND LABELS FOR THE GIVEN TRIAL
#self.cond_samples_npy = samples_npy[self.condition_number,:,:]
#self.cond_labels_npy = labels_npy[self.condition_number,:,:]
## Split data into train and test sets
#if self.test_split_users:
#    # NOT FINISHED YET
#    # Randomly pick the test_split_fraction of users to be completely held out of training to be used for testing
#    num_test_users = round(len(self.clients)*self.test_split_fraction)
#    # Pick/sample num_test_users from self.clients to be removed and put into self.testing_clients
#    self.testing_clients = [self.clients.pop(random.randrange(len(self.clients))) for _ in range(num_test_users)]
#    # Hmmm this requires a full rewrite of the code... 
#    raise ValueError("test_split_users is not fully supported yet")
#elif self.test_split_each_update:
#    # Idk this might actually be supported just in a different function. I'm not sure. Don't plan on using it rn so who cares
#    raise ValueError("test_split_each_update not supported yet.  Idk if this is necessary to add")
#else: 
#    testsplit_upper_bound = round((1-self.test_split_fraction)*(self.cond_samples_npy.shape[0]))
## Set the number of examples (used to be done on init) --> ... THIS IS ABOUT TRAIN/TEST SPLIT
#self.train_samples = testsplit_upper_bound
#self.test_samples = self.cond_samples_npy.shape[0] - testsplit_upper_bound
## The below gets stuck in the debugger and just keeps running until you step over
#train_test_update_number_split = min(self.update_ix, key=lambda x:abs(x-testsplit_upper_bound))
#self.max_training_update_upbound = self.update_ix.index(train_test_update_number_split)
my_client.test_split_idx = my_client.update_ix[my_client.max_training_update_upbound]
#############################################################################

# Do I really want this here...
my_client.local_round += 1
# Check if you need to advance the update
# ---> THIS IMPLIES THAT I AM CREATING A NEW TRAINING LOADER FOR EACH UPDATE... this is what I want actually I think
if (my_client.local_round>1) and (my_client.current_update < 16) and (my_client.local_round%my_client.local_round_threshold==0):
    my_client.current_update += 1
    print(f"Client{my_client.ID} advances to update {my_client.current_update}")
# Slice the full client dataset based on the current update number
if my_client.current_update < my_client.max_training_update_upbound:
    my_client.update_lower_bound = my_client.update_ix[my_client.current_update]
    my_client.update_upper_bound = my_client.update_ix[my_client.current_update+1]
else:
    my_client.update_lower_bound = my_client.max_training_update_upbound - 1
    my_client.update_upper_bound = my_client.max_training_update_upbound
# Set the Dataset Obj
# Uhhhh is this creating a new one each time? As long as its not re-reading in the data it probably doesn't matter...
#train_data = read_client_data(self.dataset, self.ID, self.current_update, is_train=True)  # Original code
#CustomEMGDataset(emgs_block1[my_user][condition_idx,update_lower_bound:update_upper_bound,:], refs_block1[my_user][condition_idx,update_lower_bound:update_upper_bound,:])
training_dataset_obj = CustomEMGDataset(my_client.cond_samples_npy[my_client.update_lower_bound:my_client.update_upper_bound,:], my_client.cond_labels_npy[my_client.update_lower_bound:my_client.update_upper_bound,:])
X_data = torch.Tensor(training_dataset_obj['x']).type(torch.float32)
y_data = torch.Tensor(training_dataset_obj['y']).type(torch.float32)
training_data_for_dataloader = [(x, y) for x, y in zip(X_data, y_data)]

print(f"cb load_train_data(): Client{my_client.ID}: Setting Training DataLoader")
# Set dataloader
if batch_size == None:
    batch_size = my_client.batch_size
dl = DataLoader(
    dataset=training_data_for_dataloader,
    batch_size=batch_size, 
    drop_last=False,  # Yah idk if this should be true or false or if it matters...
    shuffle=False) 
#return dl

cb load_train_data(): Client6: Setting Training DataLoader


In [93]:
for x,y in dl:
    print(x.size())
    print(y.size())

torch.Size([1200, 64])
torch.Size([1200, 2])


In [97]:
#cli6_cond0_npy = samples_npy[0]
#cli6_cond0_emg = emgs_block1[keys[my_client_num]][0]
#sum(sum(cli6_cond0_npy - cli6_cond0_emg))

print(sum(x - cli6_cond0_npy[:1200, :]))
print(sum(sum(x - cli6_cond0_npy[:1200, :])))

tensor([-2.1929e+03, -2.5532e+03,  8.6848e+02, -2.1895e+03,  6.5569e+03,
        -2.2461e+04,  1.7643e+03,  5.0437e+03,  6.9633e+03,  1.2043e+04,
         7.5770e+03,  9.5396e+03,  1.1119e+04,  8.7130e+03,  5.3621e+03,
         4.7259e+03, -3.0674e+03, -2.1818e+04, -7.4441e+03,  1.5363e+04,
         1.2195e+03, -5.2106e+03, -4.0107e+03, -6.0993e+03, -4.4521e+03,
        -1.8632e+03,  1.1350e+03, -4.3047e+02, -4.4105e+02, -1.7976e+03,
        -2.1017e+03, -3.0254e+02,  1.4371e+03,  7.8531e+03,  9.2425e+03,
         1.0661e+04,  1.3320e+04,  1.0254e+04,  1.0509e+04,  9.0886e+03,
         1.0372e+04,  5.9883e+03,  3.3941e+03, -8.9814e+03, -7.4265e+03,
         1.4346e+04,  1.4491e+04, -1.4238e+03, -2.1487e+03,  8.0973e+03,
         1.1084e+04,  2.1744e+03, -6.2367e+02, -3.7227e+03, -7.5776e+03,
        -4.4595e+03,  5.2394e+03,  5.2911e+03,  5.8159e+03,  5.0607e+03,
         1.2666e+04,  1.5031e+04,  1.1500e+04,  5.1229e-02],
       dtype=torch.float64)
tensor(176112.3001, dtype=torch.flo

In [101]:
x[:10,0]

tensor([0.0000, 0.0000, 0.9795, 0.9795, 2.5663, 2.5663, 4.0318, 4.0318, 3.9936,
        3.9936])

In [102]:
cli6_cond0_npy[:10,0]

array([10.42677364, 14.68297396, 14.68297396, 16.38068939, 16.38068939,
       16.38068939, 16.59242395,  9.08707751,  9.08707751,  9.08707751])

In [105]:
print(sum(x[0,:]))
print(sum(cli6_cond0_npy[0,:]))

tensor(0.)
510.56800753619746


In [96]:
#cli6_cond0_labels_npy = labels_npy[0]
#cli6_cond0_ref = refs_block1[keys[my_client_num]][0]
#sum(sum(cli6_cond0_labels_npy - cli6_cond0_ref))

print(sum(y - cli6_cond0_labels_npy[:1200, :]))
print(sum(sum(y - cli6_cond0_labels_npy[:1200, :])))

tensor([-2140.2960,   311.4015], dtype=torch.float64)
tensor(-1828.8945, dtype=torch.float64)


Investing training_data_for_dataloader

In [107]:
training_data_for_dataloader[0]

(tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 tensor([0., 0.]))

In [ ]:
#training_dataset_obj = CustomEMGDataset(my_client.cond_samples_npy[my_client.update_lower_bound:my_client.update_upper_bound,:], my_client.cond_labels_npy[my_client.update_lower_bound:my_client.update_upper_bound,:])
#X_data = torch.Tensor(training_dataset_obj['x']).type(torch.float32)
#y_data = torch.Tensor(training_dataset_obj['y']).type(torch.float32)
#training_data_for_dataloader = [(x, y) for x, y in zip(X_data, y_data)]

In [112]:
print(training_dataset_obj['x'].shape)
print(X_data.shape)
print()
print(training_dataset_obj['x'][:10,0])
print(X_data[:10,0])

torch.Size([1200, 64])
torch.Size([1200, 64])

tensor([0.0000, 0.0000, 0.9795, 0.9795, 2.5663, 2.5663, 4.0318, 4.0318, 3.9936,
        3.9936])
tensor([0.0000, 0.0000, 0.9795, 0.9795, 2.5663, 2.5663, 4.0318, 4.0318, 3.9936,
        3.9936])


Note taht while X_data and training_dataset_obj['x'] are consistent, they already match the x found in the dataloader and NOT the training data from my npy file... Is my npy file wrong??

In [114]:
my_client.update_lower_bound

0

In [115]:
my_client.update_upper_bound

1200

In [117]:
my_client.cond_samples_npy[:10, 0]

array([0.        , 0.        , 0.97946232, 0.97946232, 2.5663038 ,
       2.5663038 , 4.03178426, 4.03178426, 3.99362317, 3.99362317])

In [113]:
training_dataset_obj = CustomEMGDataset(my_client.cond_samples_npy[my_client.update_lower_bound:my_client.update_upper_bound,:], my_client.cond_labels_npy[my_client.update_lower_bound:my_client.update_upper_bound,:])

In [111]:
training_dataset_obj['y'].shape

torch.Size([1200, 2])

In [ ]:
#training_data_for_dataloader = [(x, y) for x, y in zip(X_data, y_data)]
for x,y in X_data, y_data:

# CONTINUED CODE

In [ ]:
print()

if self.auto_break and self.check_done(acc_lss=[self.rs_test_acc], top_cnt=self.top_cnt):
    print("Breaking")
    break

self.evaluate(train=False, test=True)
print("\nBest Loss.")
print(min(self.rs_test_loss))

for idx, client in enumerate(self.clients):
    #self.cost_func_comps_dict[idx] = client.cost_func_comps_log
    #self.gradient_dict[idx] = client.gradient_norm_log
    self.cost_func_comps_log.append(client.cost_func_comps_log)
    self.gradient_norm_log.append(client.gradient_norm_log)

self.save_results(save_cost_func_comps=True, save_gradient=True)
model_path = os.path.join("models", self.dataset)
model_path = os.path.join(model_path, "Local")
for client in self.clients:
    client.save_item(client.model, 'local_client_model', item_path=model_path)
# No idea where this global model is coming from? Why did they save it...
self.save_global_model()

## From NB 201

In [ ]:
def execute_training_loop(self):
    self.simulate_data_stream()
    self.train_model()

In [ ]:
def simulate_data_stream(self, streaming_method=True):
    if streaming_method:
        streaming_method = self.data_stream
    need_to_advance=True
    self.current_round += 1
    if self.current_update==16:  #17: previously 17 but the last update is super short so I cut it out
        #print("Maxxed out your update (you are on update 18), continuing training on last update only")
        # Probably ought to track that we maxed out --> LOG SYSTEM
        # We are stopping an update early, so use -3/-2 and not -2/-1 (the last update)
        lower_bound = (update_ix[-3] + update_ix[-2])//2  #Use only the second half of each update
        upper_bound = update_ix[-2]
        self.learning_batch = upper_bound - lower_bound
    elif streaming_method=='streaming':
        # If we pass threshold, move on to the next update
        if self.current_round%self.local_round_threshold==0:
            self.current_update += 1

            self.update_transition_log.append(self.latest_global_round)
            if self.verbose==True and self.ID==1:
                print(f"Client {self.ID}: New update after lrt passed: (new update, current global round, current local round): {self.current_update, self.latest_global_round, self.current_round}")
                print()

            # Using only the second half of each update for co-adaptivity reasons
            lower_bound = (update_ix[self.current_update] + update_ix[self.current_update+1])//2  
            upper_bound = update_ix[self.current_update+1]
            self.learning_batch = upper_bound - lower_bound
        elif self.current_round>2:
            # This is the base case
            # The update number didn't change so we don't need to overwrite everything with the same data
            need_to_advance = False
        else:
            # This is for the init case (current round is 0 or 1)
            # need_to_advance is true, so we overwrite s and such... this is fine 
            lower_bound = (update_ix[self.current_update] + update_ix[self.current_update+1])//2  
            upper_bound = update_ix[self.current_update+1]
            self.learning_batch = upper_bound - lower_bound

        self.current_update += 1
    else:
        raise ValueError(f'streaming_method ("{streaming_method}") not recognized: this data streaming functionality is not supported')

    if need_to_advance:
        s_temp = self.training_data[lower_bound:upper_bound,:]
        # First, normalize the entire s matrix
        if self.normalize_EMG:
            s_normed = s_temp/np.amax(s_temp)
        else:
            s_normed = s_temp
        # Now do PCA unless it is set to 64 (AKA the default num channels i.e. no reduction)
        # Also probably ought to find a global transform if possible so I don't recompute it every time...
        if self.PCA_comps!=self.pca_channel_default:  
            pca = PCA(n_components=self.PCA_comps)
            s_normed = pca.fit_transform(s_normed)
        s = np.transpose(s_normed)
        self.F = s[:,:-1] # note: truncate F for estimate_decoder
        v_actual = self.w@s
        p_actual = np.cumsum(v_actual, axis=1)*self.dt  # Numerical integration of v_actual to get p_actual
        p_reference = np.transpose(self.labels[lower_bound:upper_bound,:])

        #####################################################################
        # Add the boundary conditions code here
        if self.use_zvel:
            # Maneeshika code
            p_ref_lim = self.labels[lower_bound:upper_bound,:]
            if self.current_round<2:
                self.vel_est = np.zeros_like((p_ref_lim))
                self.pos_est = np.zeros_like((p_ref_lim))
                self.int_vel_est = np.zeros_like((p_ref_lim))
                self.vel_est[0] = self.w@s[:,0]  # Translated from: Ds_fixed@emg_tr[0]
                self.pos_est[0] = [0, 0]
            else:
                prev_vel_est = self.vel_est[-1]
                prev_pos_est = self.pos_est[-1]

                self.vel_est = np.zeros_like((p_ref_lim))
                self.pos_est = np.zeros_like((p_ref_lim))
                self.int_vel_est = np.zeros_like((p_ref_lim))

                self.vel_est[0] = prev_vel_est
                self.pos_est[0] = prev_pos_est
            for tt in range(1, s.shape[1]):
                # Note this does not keep track of actual updates, only the range of 1 to s.shape[1] (1202ish)
                vel_plus = self.w@s[:,tt]  # Translated from: Ds_fixed@emg_tr[tt]
                p_plus = self.pos_est[tt-1, :] + (self.vel_est[tt-1, :]*self.dt)
                # These are just correctives, such that vel_plus can get bounded
                # x-coordinate
                if abs(p_plus[0]) > 36:  # 36 hardcoded from earlier works
                    p_plus[0] = self.pos_est[tt-1, 0]
                    vel_plus[0] = 0
                    self.hit_bound += 1 # update hit_bound counter
                if abs(p_plus[1]) > 24:  # 24 hardcoded from earlier works
                    p_plus[1] = self.pos_est[tt-1, 1]
                    vel_plus[1] = 0
                    self.hit_bound += 1 # update hit_bound counter
                if self.hit_bound > 200:  # 200 hardcoded from earlier works
                    p_plus[0] = 0
                    vel_plus[0] = 0
                    p_plus[1] = 0
                    vel_plus[1] = 0
                    self.hit_bound = 0
                # now update velocity and position
                self.vel_est[tt] = vel_plus
                self.pos_est[tt] = p_plus
                # calculate intended velocity
                self.int_vel_est[tt] = calculate_intended_vels(p_ref_lim[tt], p_plus, 1/self.dt)

            self.V = np.transpose(self.int_vel_est[:tt+1])
            #print(f"V.shape: {self.V.shape}")
        else:
            # Original code
            self.V = (p_reference - p_actual)*self.dt

        
def train_given_model_1_comm_round(self, model, which):
    '''This can be used for training the in ML pipeline but principally is for local-finetuning (eg training a model after it as completed its global training pipeline).'''

    D_0 = copy.deepcopy(self.w_prev)
    # Set the w_prev equal to the current w:
    self.w_prev = copy.deepcopy(model)
    if self.global_method in ["FedAvg", "NoFL", "FedAvgSB", "Per-FedAvg", "Per-FedAvg FO", "Per-FedAvg HF"]:
        for i in range(self.num_steps):
            if self.normalize_dec:
                model /= np.amax(model)

            if self.method=='EtaGradStep':
                model = self.train_eta_gradstep(model, self.eta, self.F, model, self.H, self.V, self.learning_batch, self.alphaF, self.alphaD, PCA_comps=self.PCA_comps)
            elif self.method=='EtaScipyMinStep':
                model = self.train_eta_scipyminstep(self.w, self.eta, self.F, model, self.H, self.V, self.learning_batch, self.alphaF, self.alphaD, D_0, self.verbose, PCA_comps=self.PCA_comps)
            elif self.method=='FullScipyMinStep':
                model = self.train_eta_scipyminstep(model, self.eta, self.F, model, self.H, self.V, self.learning_batch, self.alphaF, self.alphaD, D_0, self.verbose, PCA_comps=self.PCA_comps, full=True)
            else:
                raise ValueError("Unsupported method")

            #if self.mix_in_each_steps:
            #    raise("mix_in_each_steps=True: Functionality not yet supported")
            #    self.mixed_w = self.smoothbatch*self.w + ((1 - self.smoothbatch)*self.mixed_w)

        if self.normalize_dec:
            model /= np.amax(model)

    if which!=None:
        return model, self.eval_model(which)
    else:
        return model

In [ ]:
def eval_model(self, which):
    if which=='local':
        my_dec = self.w
        my_V = self.V
    else:
        raise ValueError("Please set <which> to either local or global")
    # Just did this so we wouldn't have the 14 decimals points it always tries to give
    if self.round2int:
        temp = np.ceil(cost_l2(self.F, my_dec, self.H, my_V, self.learning_batch, self.alphaF, self.alphaD))
        # Setting to int is just to catch overflow errors
        # For RT considerations, ints are also generally ints cheaper than floats...
        out = int(temp)
    else:
        temp = cost_l2(self.F, my_dec, self.H, my_V, self.learning_batch, self.alphaF, self.alphaD, Ne=self.PCA_comps)
        out = round(temp, 3)
    return out

def test_inference(self, test_current_dec=True):
    ''' No training / optimization, this just tests the fed in dec '''

    if test_current_dec==True:
        test_dec = self.w
    else:
        #test_dec is whatever you input, presumably a matrix... probably should check
        test_dec = test_current_dec
        if np.prod(test_dec.shape)!=(self.PCA_comps*2):
            raise ValueError(f"Unexpected size of test_current_dec: {np.prod(test_dec.shape)} vs {self.PCA_comps*2} expected")

    # This sets FVD using the full client dataset
    # Since we aren't doing any optimization then it shouldn't matter if we use updates or not...
    simulate_data_stream(streaming_method='full_data')
    # Evaluate cost
    temp = cost_l2(self.F, test_dec, self.H, self.V, self.learning_batch, self.alphaF, self.alphaD, Ne=self.PCA_comps)
    dec_cost = round(temp, 3)
    # Also want to see actual output 
    # This might be the cost and not the actual position...
    D_reshaped = np.reshape(test_dec,(2,self.PCA_comps))
    dec_pos = D_reshaped@self.F + self.H@self.V[:,:-1] - self.V[:,1:]
    return dec_cost, dec_pos

In [ ]:
assert(False)

In [ ]:
import h5py

In [ ]:
my_list = [(1,2,3), (3,2,4), (1,1,1), (234,345,2345345), (34132412413,1,1324)]
my_list2 = [(1,2,3), (3,2,4), [], (234,345,2345345), (34132412413,1,1324)]

In [ ]:
#base_data_path = 'C:\\Users\\kdmen\\Desktop\\Research\\personalization-privacy-risk\\Data\\Client_Specific_Files\\'

with h5py.File('C:\\Users\\kdmen\\Downloads\\hdf5_test1.h5', 'w') as hf:
    #hf.create_dataset('gradient_norm_log', data=self.gradient_norm_log)
    #hf.create_dataset('my_list', my_list)
    G1 = hf.create_group('by_client')
    for i in range(len(my_list)):
        name_str = 'ClientID' + str(i)
        G1.create_dataset(name_str, data=my_list[i])
    

In [ ]:
with h5py.File('C:\\Users\\kdmen\\Downloads\\hdf5_test1.h5', 'r') as hf:
    base_items = list(hf.items())
    print(base_items)
    print()
    G1 = hf.get('by_client')
    print(G1)
    print()
    G1_items = list(G1.items())
    print(G1_items)
    print()
    print(type(G1_items[0]))
    print()
    print(G1_items[0])
    print()
    print(np.array(G1.get('ClientID0')))
    print(type(np.array(G1.get('ClientID0'))))
    print(len(np.array(G1.get('ClientID0'))))
    print(np.array(G1.get('ClientID0'))[0])

In [ ]:
cost_func_comps_log = [[], [(66.26913452148438, 0.0006768530583940446, 0.0)], [(2.9157848358154297, 0.0006768530583940446, 0.0), (60.782691955566406, 0.0007280856370925903, 0.0)], [], [(20.13532257080078, 0.0006768530583940446, 0.0)], [(51.6102180480957, 0.0006768530583940446, 0.0), (79754.7734375, 0.018196912482380867, 0.0)], [], [], [], [(2.3531734943389893, 0.0006768530583940446, 0.0), (138.36375427246094, 0.0007864069775678217, 0.0), (13262.6416015625, 0.012903410941362381, 0.0)], [(68.56493377685547, 0.0006768530583940446, 0.0), (296026.5625, 0.04193491116166115, 0.0)], [(61.63677978515625, 0.0006768530583940446, 0.0)], [], []]

In [ ]:
with h5py.File('C:\\Users\\kdmen\\Downloads\\hdf5_cfc_log.h5', 'w') as hf:
    G1 = hf.create_group('by_client')
    for idx, list_of_tuples in enumerate(cost_func_comps_log):
        name_str = 'ClientID' + str(idx)
        print(name_str)
        G1.create_dataset(name_str, data=list_of_tuples)